In [2]:
# Various Imports

import pandas as pd
import numpy as np
import os
import cv2
import timm
import time
import torch
import torchvision
import wandb
import hiplot
import neptune.new as neptune
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Preprocessing Data 

for i in tqdm(range(0, 1), desc ="Pre-processing Data", colour="#35AF92"):
    data=pd.read_csv("D:\\python\\CIS473\\CIS-473-Machine-Learning\\credit_data\\application_train.csv",skiprows=1).dropna(axis=1)
    data=pd.get_dummies(data)
    sdtr=StandardScaler()
    start_row=100000
    end_row=100000
    x_train,x_test,y_train,y_test=train_test_split(data.iloc[:,2:],data.iloc[:,1],test_size=0.2,random_state=0)
        
    x_train=sdtr.fit_transform(x_train)
    x_test=sdtr.transform(x_test)

    y_train=y_train.to_numpy()
    y_test=y_test.to_numpy()

    x_train=torch.from_numpy(x_train.astype(np.float32))
    y_train=torch.from_numpy(y_train)
    x_test=torch.from_numpy(x_test.astype(np.float32))
    y_test=torch.from_numpy(y_test)

    y_train=y_train.view(y_train.shape[0],1)
    y_test=y_test.view(y_test.shape[0],1)

Pre-processing Data: 100%|███████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.58s/it]


In [4]:
# Defining Dataset

class CreditDataset(Dataset):
    def __init__(self,x,y):   
        self.x=x
        self.y=y
        
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.x.shape[0]
    
print(x_train.shape)

torch.Size([246008, 142])


In [5]:
# Loading Data

for i in tqdm(range(0, 1), desc ="Loading Data", colour="#26B69C"):
    train=CreditDataset(x_train,y_train)
    train_data=DataLoader(dataset=train,batch_size=1)
    test=CreditDataset(x_test,y_test)
    test_data=DataLoader(dataset=test,batch_size=1)

Loading Data: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2002.05it/s]


In [6]:
# Defining Model

class CreditNet(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.hidden_size1=500
        self.hidden_size2=800
        self.hidden_size3=200
        self.input_size=input_size
        self.output_size=output_size
        self.linear1=nn.Linear(self.input_size,self.hidden_size1)
        self.linear2=nn.Linear(self.hidden_size1,self.hidden_size2)
        self.linear3=nn.Linear(self.hidden_size2,self.hidden_size3)
        self.linear4=nn.Linear(self.hidden_size3,self.output_size)
        
    def forward(self,x_train):
        out=self.linear1(x_train)
        out=F.relu(out)
        out=self.linear2(out)
        out=F.relu(out)
        out=self.linear3(out)
        out=F.relu(out)
        out=self.linear4(out)
        out=torch.sigmoid(out)
        return out

In [7]:
# Model Parameters

device=torch.device("cuda")
nepochs=10
input_size=x_train.shape[1]
output_size=2
learning_rate=0.01

model=CreditNet(input_size,output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
LossFunction = nn.CrossEntropyLoss()

In [ ]:
#Traning Model

loss_history = []
for epoch in tqdm(range(nepochs),desc="Epoch",colour="#26B69C"):
    for data,label in tqdm(train,colour="#26B69C",mininterval=20):
        optimizer.zero_grad()
        data = data.to(device)
        label = label.to(device)
        out = model(data).view(1,-1)
        #print(out,label)
        loss = LossFunction(out,label)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
    print(out,label)
    print(f"Epoch {epoch}: loss: {loss.item()}")

Epoch:  10%|███████▌                                                                   | 1/10 [03:10<28:33, 190.35s/it]

tensor([[9.9995e-01, 4.6802e-05]], device='cuda:0', grad_fn=<ViewBackward0>) tensor([0], device='cuda:0')
Epoch 0: loss: 0.3132866621017456



Epoch:  20%|███████████████                                                            | 2/10 [06:38<26:48, 201.04s/it]

tensor([[9.9998e-01, 2.1922e-05]], device='cuda:0', grad_fn=<ViewBackward0>) tensor([0], device='cuda:0')
Epoch 1: loss: 0.3132734000682831



Epoch:  30%|██████████████████████▌                                                    | 3/10 [10:05<23:46, 203.74s/it]

tensor([[9.9999e-01, 1.4307e-05]], device='cuda:0', grad_fn=<ViewBackward0>) tensor([0], device='cuda:0')
Epoch 2: loss: 0.31326931715011597



 74%|█████████████████████████████████████████████████████▍                  | 182378/246008 [02:40<00:53, 1186.50it/s]

In [ ]:
correct=0
with torch.no_grad():
    for data,label in tqdm(test,colour="#26B69C",mininterval=20):
        data = data.to(device)
        label=label.to(device)
        out=model(data)
        pred=torch.argmax(out)
        if pred==label:
            correct+=1
print("Accuracy:",round(correct/len(test),3))

In [ ]:
FILE="D:\\python\\CIS473\\CIS-473-Machine-Learning\\credit_data\\models\\model.pth"
torch.save(model,FILE)